In [5]:
from scapy.all import *
from pprint import pprint
import sys
import numpy as np
import os
import dpkt
import matplotlib.pyplot as plt

MAX_LINE_RATE =10e9
PACKET_SIZE = 1034


In [6]:
def read_pcap(out_dir, dst_mac_is_ts = True, try_compare_counters = True):
    
    if try_compare_counters:
        count_file = os.path.join(out_dir, 'counters_0.out')
        for line in open(count_file):
            if line.startswith('5/0'):
         #       print("counters_0.out successfully opened")
                n_pkts_str = line.split()[-1]
                n_start_pkts = int(n_pkts_str[:-1])
        count_file = os.path.join(out_dir, 'counters_1.out')
        for line in open(count_file):
            if line.startswith('5/0'):
          #      print("counters_1.out successfully opened")
                n_pkts_str = line.split()[-1]
                n_end_pkts = int(n_pkts_str[:-1])   
        n_expected_pkts = (n_end_pkts - n_start_pkts)
        
        #print("Expecting {} packets".format(n_expected_pkts))
    
    
    pcf = os.path.join(out_dir, 'moongen', 'moongen.pcap')
    
    counters = {
        'ingress': 0,
        'egress': 0,
    }

    total_osize = 0
    total_isize = 0
    pkt_othroughput = []
    pkt_ithroughput = []
    pkt_ithroughput.append(1000)
    pkt_ithroughput.append(2000)
    pkt_othroughput.append(1000)
    pkt_othroughput.append(2000)
    
    f = open(pcf, 'rb')
    #print("Reading {}".format(pcf))
    pcap=dpkt.pcap.Reader(f)
    
    for ts, buf in pcap:
        try:
            eth=dpkt.ethernet.Ethernet(buf)
            ip=eth.data
            
            #TODO: Add a check to distinguish ingress and egress packets
            # If ingress, add to input array.
            # If egress, add to output array
            
            if eth.src == b'\x00\x00\x00\x00\x00\x02':
                counters['ingress'] += 1
            else:
                counters['egress'] += 1

            if dst_mac_is_ts:
                x3, x2, x1 = struct.unpack("!HHH", eth.dst)
                ts = float((x3 << 32) | (x2 << 16) | x1) * 1e-9
                if eth.src == b'\x00\x00\x00\x00\x00\x01':
                    if ts <= pkt_othroughput[0]:
                        pkt_othroughput[0] = ts
                    pkt_othroughput[1] = ts
                    total_osize += PACKET_SIZE
                
                elif eth.src == b'\x00\x00\x00\x00\x00\x02':
                    if ts <= pkt_ithroughput[0]:
                        pkt_ithroughput[0] = ts
                    pkt_ithroughput[1] = ts  
                    total_isize += PACKET_SIZE

        except Exception as e:
            print("Exception parsing packet {} : {}".format(buf, e))
            Ether(buf).show2()
            raise
    ''' 
    print("Total input bytes: " + str(total_isize))
    print("Total output bytes: " + str(total_osize))
    print("Input throughput start, end: " + str(pkt_ithroughput))
    print("Output start, end " + str(pkt_othroughput))
    ''' 
    in_tput = ((total_isize * 8) / (pkt_ithroughput[1] - pkt_ithroughput[0]))
    out_tput = ((total_osize * 8) / (pkt_othroughput[1] - pkt_othroughput[0]))
    
    ipps = (counters['ingress'] / (pkt_ithroughput[1] - pkt_ithroughput[0]))
    opps = (counters['egress'] / (pkt_othroughput[1] - pkt_othroughput[0]))
    ''' 
    print("Input throughput " + str(in_tput))
    print("Output throughput " + str(out_tput))
    
    print("Input pps: %d" %(ipps))
    print("Output pps: %d" %(opps))
    ''' 
    return in_tput, out_tput

In [12]:
linerate = [10.00, 50.00, 70.00, 100.00]
#linerate = [90.00]
for rate in linerate:
    tot_itput = 0
    tot_otput = 0
    tot_latency = 0
    all_tput = []
    i = 1
    
    while i <= 5:
        out_dir = "/home/ncshy/P4Boosters/testing/fec/execution/test_multiple_run/fpga_decoder_rerun_%d/fpga_decoder_rerun_%d_%.2f"%(i,i,rate)
        #read_pcap_with_pkt(out_dir, True, True)
        #Set compress to True for Compressor and False for decompressor
        compress = True
        tput = read_pcap(out_dir, True, True)
        all_tput.append(tput)
        #latency = get_latency(out_dir, True)
        #all_latency.append(latency) 
        #if i == 2:
         #   i += 2
          #  continue
        i += 1
        
    count = 0
    for item in all_tput:
        tot_itput += item[0]
        tot_otput += item[1]
        count += 1
    avg_itput = tot_itput/count
    avg_otput = tot_otput/count
    print("fpga_fec_encoder_rerun_%.2f tput"%(rate))
    print(avg_itput)
    print(avg_otput)


fpga_fec_encoder_rerun_90.00 tput
8629817976.775293
7192286272.4526205
